In [2]:
!pip install tqdm

In [3]:
import numpy as np
import tifffile
import tomopy
import matplotlib.pyplot as plt
from tqdm import tqdm

In [4]:
# Load the TIFF stack
tiff_path = "/home/minyu/recon_025_AS1.6_4DPF_WB_18x(361nm).tiff"  # or "your_file.tif" for multipage
stack = tifffile.imread(tiff_path).astype(np.float32)

print("Stack shape:", stack.shape)
stack /= np.max(stack)

print("Example slice shape:", stack[0].shape)


Stack shape: (2160, 1855, 1719)
Example slice shape: (1855, 1719)


In [ ]:
stack = stack.astype(np.float32)
stack /= np.max(stack)
corrected_stack = np.zeros_like(stack)

# Loop with progress bar and error handling
for i in tqdm(range(stack.shape[0]), desc="Removing ring artifacts"):
    try:
        corrected = tomopy.prep.stripe.remove_stripe_fw(
            stack[i], level=2, wname='db2', sigma=1, pad=True
        )

        # Catch total zeroed slices
        if corrected is None or np.all(corrected == 0):
            print(f"⚠️ Slice {i} returned None or zeros.")
            corrected = stack[i]

        corrected_stack[i] = corrected

    except Exception as e:
        print(f"❌ Error on slice {i}: {e}")
        corrected_stack[i] = stack[i]


Removing ring artifacts:   0%|                                                         | 1/2160 [00:00<22:53,  1.57it/s]

❌ Error on slice 0: not enough values to unpack (expected 3, got 2)


Removing ring artifacts:   0%|                                                         | 2/2160 [00:01<20:12,  1.78it/s]

❌ Error on slice 1: not enough values to unpack (expected 3, got 2)


Removing ring artifacts:   0%|                                                         | 3/2160 [00:01<20:25,  1.76it/s]

...

Removing ring artifacts:   0%|▏                                                        | 7/2160 [00:03<18:50,  1.90it/s]

❌ Error on slice 6: not enough values to unpack (expected 3, got 2)


Removing ring artifacts:   0%|▏                                                        | 8/2160 [00:04<20:05,  1.78it/s]

❌ Error on slice 7: not enough values to unpack (expected 3, got 2)


Process ForkPoolWorker-221:
Process ForkPoolWorker-211:
Process ForkPoolWorker-219:
Process ForkPoolWorker-220:
Process ForkPoolWorker-218:
Exception ignored in: <function _releaseLock at 0x7fc309af4550>
Traceback (most recent call last):
  File "/home/minyu/miniconda3/envs/tomo-env/lib/python3.9/logging/__init__.py", line 227, in _releaseLock
    def _releaseLock():
KeyboardInterrupt: 
Process ForkPoolWorker-222:
Process ForkPoolWorker-210:
Process ForkPoolWorker-216:
Process ForkPoolWorker-217:
Process ForkPoolWorker-215:
Process ForkPoolWorker-209:
Process ForkPoolWorker-206:
Process ForkPoolWorker-213:
Process ForkPoolWorker-208:
Process ForkPoolWorker-205:
Process ForkPoolWorker-207:
Process ForkPoolWorker-203:
Process ForkPoolWorker-202:
Process ForkPoolWorker-214:
Process ForkPoolWorker-204:
Process ForkPoolWorker-212:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent

In [ ]:
mid = stack.shape[0] // 2
print(f"Original slice min/max: {stack[mid].min()} / {stack[mid].max()}")
print(f"Corrected slice min/max: {corrected_stack[mid].min()} / {corrected_stack[mid].max()}")

In [ ]:
diff = stack[mid] - corrected_stack[mid]
plt.imshow(diff, cmap='gray')
plt.title("Difference (Original - Corrected)")
plt.colorbar()
plt.show()

In [ ]:
mid = stack.shape[0] // 2

plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.imshow(stack[mid], cmap='gray', vmin=0, vmax=1)
plt.title("Original"); plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(corrected_stack[mid], cmap='gray', vmin=0, vmax=1)
plt.title("Corrected"); plt.axis('off')

plt.tight_layout()
plt.show()
